<h2> Automatic filtering </h2> 

In [ ]:
# import internal .py files
import file_path_management as fpath
import public_library as plib

In [ ]:
# import packages
import csv
import pandas as pd
import re
from bs4 import BeautifulSoup
import requests
import os
import numpy as np
import string

<h3> Parameters: </h3> 

In [ ]:
# on-topic keyword lexicon
on_topic_kws = [
    'thalamocortical', 'thalamo-cortical', 'corticothalamic', 'cortico-thalamic',
    'tracing', 'tracer', 'tract tracing', 'tract-tracing', 'axonal tracing', 'neural tracing', 'anatomical tracing', 'neuroanatomical tracing',
    'thalamus', 'cortex', 'thalamic', 'cortical', "staining", "dye", 
    'connection', 'projection', 'connectivity', 'connectome', "anterograde", "retrograde", "injection", "injected", "injecting", "inject"]

on_topic_kws_weights = {
    'tracing': 10, 'tracer': 10, 'tract tracing': 10, 'tract-tracing': 10, 'axonal tracing': 10, 'neural tracing': 10, 'anatomical tracing': 10, 'anatomical neural tracing': 10,
    'thalamocortical': 5, 'thalamo-cortical': 5, 'corticothalamic': 5, 'cortico-thalamic': 5,
    'connection': 2, 'projection': 2, 'connectivity': 2, 
    'thalamus': 1, 'cortex': 1, 'thalamic': 1, 'cortical': 1, 
    'connectome': 1}

# pay attention to false negative, false positive cases

# ChatGPT, queries for relatedness of topic
ChatGPT_related_queries = ['Does the given text include information of thalamocotical connection?',
                           'Does this paper provide data of thalamocotical connection?',
                           'Does the given text include information of connection between thalamus and cortex?']

<h3> Predefined fucntions: </h3> 

In [ ]:
# count the number of times that certain on-topic keyword appear in a given text
def count_keyword(text, keyword):
    # remove non-alphabetic characters but keep spaces
    # punc_marks = list(string.punctuation) # python punctuations
    # print(punc_marks)
    # for punc in punc_marks:
    #     if punc in text:
    #         text = text.replace(punc, '')
    text = ''.join(e for e in text if (e.isalpha() or e == " "))
    print(text)
    text = text.strip().lower()
    print(text)
    words = text.split()
    print(words)
    
    word_count = 0
    for word in words:
        # print(word)
        if word == keyword:
            word_count += 1
    return word_count
# --------------------start of test code--------------------
# text = 'This apple 6i7s very tasty？、  2but th&e banana this is not delicious at Is all.6'
# keyword = 'is'
# count = count_keyword(text, keyword)
# print(count)
# ---------------------end of test code---------------------

In [ ]:
# count the number of times all on-topic keywords appear in the text
# extracted from the given url
def count_freq_from_liter(text, on_topic_kws, type):
    print(text)
    text_length = len(text)
    keywords_count_fre = {}
    # count the on-topic keywords
    for i in range(len(on_topic_kws)):
        word_count = count_keyword(text, on_topic_kws[i])
        if type == "count":
            keywords_count_fre[on_topic_kws[i]] = word_count
        elif type == "frequency":
            keywords_count_fre[on_topic_kws[i]] = word_count/text_length
    return keywords_count_fre
# --------------------start of test code--------------------
# text = 'Vision for action: thalamic and cortical inputs to the macaque superior parietal lobule The dorsal visual stream, the cortical circuit that in the primate brain is mainly dedicated to the visual control of actions, is split into two routes, a lateral and a medial one, both involved in coding different aspects of sensorimotor control of actions. The lateral route, named "lateral grasping network", is mainly involved in the control of the distal part of prehension, namely grasping and manipulation. The medial route, named "reach-to-grasp network", is involved in the control of the full deployment of prehension act, from the direction of arm movement to the shaping of the hand according to the object to be grasped. In macaque monkeys, the reach-to-grasp network (the target of this review) includes areas of the superior parietal lobule (SPL) that hosts visual and somatosensory neurons well suited to control goal-directed limb movements toward stationary as well as moving objects. After a brief summary of the neuronal functional properties of these areas, we will analyze their cortical and thalamic inputs thanks to retrograde neuronal tracers separately injected into the SPL areas V6, V6A, PEc, and PE. These areas receive visual and somatosensory information distributed in a caudorostral, visuosomatic trend, and some of them are directly connected with the dorsal premotor cortex. This review is particularly focused on the origin and type of visual information reaching the SPL, and on the functional role this information can play in guiding limb interaction with objects in structured and dynamic environments. Area PEc; Area V6; Area V6A; Dorsal visual stream; Goal-directed arm movement; Sensorimotor integration.'
# keywords_count_fre = count_freq_from_liter(text, on_topic_kws)
# print(keywords_count_fre)
# ---------------------end of test code---------------------

In [ ]:
def calcul_related(info_count, related_kws_weights):
    weight = 0
    for key, value in info_count.items():
        weight += info_count[key] * related_kws_weights[key]
    return weight
# --------------------start of test code--------------------
# info_count = {'thalamocortical': 2, 'thalamo-cortical': 2, 'corticothalamic': 0, 'cortico-thalamic': 1,
#               'tracing': 0, 'tracer': 0, 'tract tracing': 0, 'tract-tracing': 0, 'axonal tracing': 1, 'neural tracing': 1, 'anatomical tracing': 1, 'anatomical neural tracing': 0,
#               'connection': 1, 'projection': 2, 'connectivity': 0, 'connectome': 0, 
#               'thalamus': 0, 'cortex': 4, 'thalamic': 2, 'cortical': 3}
# weight = calcul_related(info_count, related_kws_weights)
# print(weight)
# ---------------------end of test code---------------------


In [ ]:
# scan each url in list_of_literature_urls.txt and record information and download pdf
def extract_text:
    pass
def scan_download_record(on_topic_kws, pdf_folder_path):
    # scan each row in the potential related literature and extract information
    df = pd.read_csv(fpath.poten_litera_csv, sep = ",")
    for ind in df.index:
        if df["PMID"][ind] is not NaN:
            pmid = df["PMID"][ind]
            url = "https://pubmed.ncbi.nlm.nih.gov/" + pmid + "/"
            print(url)
            response = requests.get(url, headers = plib.headers)
            if response.status_code != 200:
                raise Exception("Error when request webpages!")
            soup = BeautifulSoup(response.content, "lxml")
            l = soup.find_all("a", {"class: id-link"}, {"data-ga-action": "DOI"})
            if(len(l) != 0):
                # print(l[0].get_text().strip())
                doi = l[0].get_text().strip()
            else:
                doi = NaN
            l = soup.find_all({"class": "current-id"}, {"title": "PubMed ID"})
            if(len(l) != 0):
                # print(l[0].get_text().strip())
                pmcid = l[0].get_text().strip()
            else:
                doi = NaN
            title = soup.find_all("h1", {"class": "heading-title"})[0].get_text().strip()
            abstract = soup.find_all("div", {"class": "abstract-content selected"})[0].get_text().strip()
            keywords = soup.find_all("p", {"class": "sub-title"})[0].get_text().strip()

            # extract title, abstract, keywords, introduction from the returned html file
            # count keywords from title + abstract + keywords
            # and make sure the process the text so that only punctuation marks and numbers are removed
            text = title + " " + title + " " + "abstract" + " " + "keywords"
            text = text.strip()
            text = re.sub(' +', ' ', text)
            text = re.sub(r"[^a-zA-Z' ']", "", text).lower()
    
            # record the information into json
            info_json = {}
            info_json['DOI'] = doi
            info_json['PMID'] = pmid
            info_json['PMCID'] = pmcid
            info_json['title'] = title
            info_json['abstract'] = abstract
            info_json['keywords'] = keywords
            info_count = count_freq_from_liter(text, on_topic_kws, type = "count")
            info_json = {**info_json, **info_count}
            weight = calcul_related(info_count, related_kws_weights)
            info_json["weight"] = weight
            print(info_json)
            columns = ['DOI', "PMID", "PMCID", 'title', "abstract", "keywords", "weight"] + on_topic_kws
            plib.add_row_to_csv(fpath.auto_filtered, info_json, columns)
# --------------------start of test code--------------------
# scan_record_download(path_urls, on_topic_kws, pdf_folder_path)
# ---------------------end of test code---------------------
# --------------------start of test code--------------------
# source_path = fpath.poten_litera_filled
# output_path = fpath.filtered_poten_litra
# plib.clear_file(output_path)
# ---------------------end of test code---------------------

# --------------------start of test code--------------------
# # merge all search results
# identifier = ["DOI", "PMID", "PMCID"]
# merge_remove_dupli(input_path, output_path)
# ---------------------end of test code---------------------

# --------------------start of test code--------------------
# df = pd.read_csv(output_path, header=None, sep=',')
# print(df.head(3))
# print(df.shape)
# ---------------------end of test code---------------------

In [ ]:
def weight_and_rank(on_topic_kws_weights):
    # weight formula

    # rank

    print("Weighting and ranking the potentially related literature succeded!")
    print("Enjoy reading!")
# --------------------start of test code--------------------
# test code
# ---------------------end of test code---------------------

<h3> Main program: </h3> 

In [ ]:
# # step 1: search for text of the literature: title, abstract, keywords
# input_path = fpath.
# output_path = 

# # clear file
# plib.clear_file(output_path)

# # extract and filling text
# text_filling(input_path, output_path, on_topic_kws, start, end)

In [ ]:
# # step 2: assign weight to each literature and rank them
# weight_and_rank(related_kws_weights)

<h3> Next step: manually read papers and find all actually related literature </h3>